# What is a Bayes net?

# Representing a Bayes net in code

# Visualising a Bayes net

# Specifying the joint probability distribution via conditional probabilities

# Parsing conditional probabilities to initialise a Bayes net

# Simulating a Bayes net

# Visualising results

# Inference from simulation

# Testing the implementation

# Extending the implementation